In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import gzip
import json
import os

import pandas as pd
from sig2kmer import read_hash_csv
from tqdm import tqdm

## Read Uniprot data

In [51]:
with gzip.open(
    "/Users/olgabot/botryllus-data/2024-04-16__uniprotkb_ZNF292_OR_RNMT_OR_BMP3_OR_CFA__botryllus_bhf_matches.json.gz"
) as f:
    uniprot_data = json.load(f)

## Import botryllus sequence

In [52]:
botryllus_dir = "/Users/olgabot/botryllus/data/botryllus-proteins/"

botryllus_sigfile = os.path.join(botryllus_dir, "Bs_proteins.fa.hp.k24.scale5.sig")

### Read botryllus kmer hashes

In [53]:
hash_csv = os.path.join(
    botryllus_dir, "Bs_proteins.fa.hp.k24.scale5.aggregated.kmers.csv"
)
botryllus_kmer_hashes = read_hash_csv(hash_csv)
# botryllus_kmer_hashes.index = botryllus_kmer_hashes.index.astype(str)
botryllus_kmer_hashes

,kmer_in_sequence,kmer_in_alphabet,hashval,read_name,filename
0,VHDTEQLLAQGHHEEETECGKYGK,hppppphhhphpppppppphphhp,1036020595944595459,BHF,Bs_proteins.fa.gz
1,TEQLLAQGHHEEETECGKYGKLPE,ppphhhphpppppppphphhphhp,2993707203445337902,BHF,Bs_proteins.fa.gz
2,EQLLAQGHHEEETECGKYGKLPEK,pphhhphpppppppphphhphhpp,973051056292048589,BHF,Bs_proteins.fa.gz
3,ETECGKYGKLPEKGSECKKHGIFC,pppphphhphhpphpppppphhhp,385192726330471768,BHF,Bs_proteins.fa.gz
4,KYGKLPEKGSECKKHGIFCRILTA,phhphhpphpppppphhhpphhph,1194029314525711201,BHF,Bs_proteins.fa.gz
...,...,...,...,...,...
303186,LEREIEAQQLVMLEAEIDCLQRRF,hppphphpphhhhphphpphppph,1833930335722081195,g72096.t1 frame:1,Bs_proteins.fa.gz
303187,EREIEAQQLVMLEAEIDCLQRRFE,ppphphpphhhhphphpphppphp,616964295969867237,g72096.t1 frame:1,Bs_proteins.fa.gz
303188,MLEAEIDCLQRRFEDMESTWISRI,hhphphpphppphpphppphhpph,1869927996039995699,g72096.t1 frame:1,Bs_proteins.fa.gz
303189,EIDCLQRRFEDMESTWISRIDGTS,phpphppphpphppphhpphphpp,995001100309400823,g72096.t1 frame:1,Bs_proteins.fa.gz


### Subset to BHF

In [54]:
bhf_hashes = botryllus_kmer_hashes.query('read_name == "BHF"')
bhf_hashes.head()

,kmer_in_sequence,kmer_in_alphabet,hashval,read_name,filename
0,VHDTEQLLAQGHHEEETECGKYGK,hppppphhhphpppppppphphhp,1036020595944595459,BHF,Bs_proteins.fa.gz
1,TEQLLAQGHHEEETECGKYGKLPE,ppphhhphpppppppphphhphhp,2993707203445337902,BHF,Bs_proteins.fa.gz
2,EQLLAQGHHEEETECGKYGKLPEK,pphhhphpppppppphphhphhpp,973051056292048589,BHF,Bs_proteins.fa.gz
3,ETECGKYGKLPEKGSECKKHGIFC,pppphphhphhpphpppppphhhp,385192726330471768,BHF,Bs_proteins.fa.gz
4,KYGKLPEKGSECKKHGIFCRILTA,phhphhpphpppppphhhpphhph,1194029314525711201,BHF,Bs_proteins.fa.gz


## Import human sequences

### Read ZNF292 kmer hashes

[O60281](https://www.uniprot.org/uniprotkb/O60281/feature-viewer?loadFeatures=true)is the primary entry for ZNF292 Human.

In [55]:
znf292_csv = "/Users/olgabot/protein-job-search/2024-04-16--botryllus-known-matches/O60281__ZN292_HUMAN.csv.gz"

In [56]:
znf292_hashes = read_hash_csv(znf292_csv)
znf292_hashes

,hashval,kmer_in_sequence,kmer_in_alphabet,start,accession,uniprotkb_id,organism_scientific_name,organism_common_name,organism_taxonid,gene_name
0,13638207749623269045,MADEEAEQERLSCGEGGCVAELQR,hhppphpppphpphphhphhphpp,0,O60281,ZN292_HUMAN,Homo sapiens,Human,9606,ZNF292
1,8404708278310155067,ADEEAEQERLSCGEGGCVAELQRL,hppphpppphpphphhphhphpph,1,O60281,ZN292_HUMAN,Homo sapiens,Human,9606,ZNF292
2,8048618853620713690,DEEAEQERLSCGEGGCVAELQRLG,ppphpppphpphphhphhphpphh,2,O60281,ZN292_HUMAN,Homo sapiens,Human,9606,ZNF292
3,17517879030661681686,EEAEQERLSCGEGGCVAELQRLGE,pphpppphpphphhphhphpphhp,3,O60281,ZN292_HUMAN,Homo sapiens,Human,9606,ZNF292
4,17298076713827473283,EAEQERLSCGEGGCVAELQRLGER,phpppphpphphhphhphpphhpp,4,O60281,ZN292_HUMAN,Homo sapiens,Human,9606,ZNF292
...,...,...,...,...,...,...,...,...,...,...
2695,5603276598621902447,KLEVHSNDPDMSVMKDISIGKATG,phphpppphphphhpphphhphph,2695,O60281,ZN292_HUMAN,Homo sapiens,Human,9606,ZNF292
2696,2918542526153815211,LEVHSNDPDMSVMKDISIGKATGR,hphpppphphphhpphphhphphp,2696,O60281,ZN292_HUMAN,Homo sapiens,Human,9606,ZNF292
2697,17814690177903370705,EVHSNDPDMSVMKDISIGKATGRG,phpppphphphhpphphhphphph,2697,O60281,ZN292_HUMAN,Homo sapiens,Human,9606,ZNF292
2698,14562833835520679758,VHSNDPDMSVMKDISIGKATGRGQ,hpppphphphhpphphhphphphp,2698,O60281,ZN292_HUMAN,Homo sapiens,Human,9606,ZNF292


In [57]:
accession = "O60281"

entry = [x for x in uniprot_data["results"] if x["primaryAccession"] == accession][0]
entry.keys()

dict_keys(['entryType', 'primaryAccession', 'secondaryAccessions', 'uniProtkbId', 'entryAudit', 'annotationScore', 'organism', 'proteinExistence', 'proteinDescription', 'genes', 'comments', 'features', 'keywords', 'references', 'uniProtKBCrossReferences', 'sequence', 'extraAttributes'])

In [58]:
znf292_hashes_in_bhf = znf292_hashes.loc[znf292_hashes.hashval.isin(bhf_hashes.hashval)]
znf292_hashes_in_bhf

,hashval,kmer_in_sequence,kmer_in_alphabet,start,accession,uniprotkb_id,organism_scientific_name,organism_common_name,organism_taxonid,gene_name
606,1452226584929511062,RRLGRPPKITTTNENQKTNTVAKQ,pphhphhphppppppppppphhpp,606,O60281,ZN292_HUMAN,Homo sapiens,Human,9606,ZNF292
609,791636867468234754,GRPPKITTTNENQKTNTVAKQEQR,hphhphppppppppppphhppppp,609,O60281,ZN292_HUMAN,Homo sapiens,Human,9606,ZNF292


In [59]:
ksize = len(znf292_hashes_in_bhf.kmer_in_sequence.iloc[0])
ksize

24

### get the full size from using the k-mer size

In [60]:
znf292_hashes_in_bhf_start = znf292_hashes_in_bhf.start.min()
znf292_hashes_in_bhf_end = znf292_hashes_in_bhf.start.max() + ksize
print(f"from {znf292_hashes_in_bhf_start}-{znf292_hashes_in_bhf_end}")

from 606-633


In [61]:
def get_feature_start(feature):
    return feature["location"]["start"]["value"]


def get_feature_end(feature):
    return feature["location"]["end"]["value"]


def get_overlapping_features(
    entry,
    kmer_start,
    kmer_end,
    ignore_types=("Natural variant", "Alternative sequence"),
    verbose=False,
):
    for feature in entry["features"]:
        start = get_feature_start(feature)
        end = get_feature_end(feature)
        feature_length = start - end
        if start >= kmer_start and start <= kmer_end:
            overlap_size = kmer_end - start
            if verbose:
                print(f"overlap size: {overlap_size}")
            if not feature["type"] in ignore_types:
                yield feature


list(
    get_overlapping_features(
        entry, znf292_hashes_in_bhf_start, znf292_hashes_in_bhf_end
    )
)

[{'type': 'Region',
  'location': {'start': {'value': 608, 'modifier': 'EXACT'},
   'end': {'value': 633, 'modifier': 'EXACT'}},
  'description': 'Disordered',
  'evidences': [{'evidenceCode': 'ECO:0000256',
    'source': 'SAM',
    'id': 'MobiDB-lite'}]},
 {'type': 'Compositional bias',
  'location': {'start': {'value': 614, 'modifier': 'EXACT'},
   'end': {'value': 633, 'modifier': 'EXACT'}},
  'description': 'Polar residues',
  'evidences': [{'evidenceCode': 'ECO:0000256',
    'source': 'SAM',
    'id': 'MobiDB-lite'}]}]

In [62]:
entry["sequence"]

{'value': 'MADEEAEQERLSCGEGGCVAELQRLGERLQELELQLRESRVPAVEAATDYCQQLCQTLLEYAEKWKTSEDPLPLLEVYTVAIQSYVKARPYLTSECENVALVLERLALSCVELLLCLPVELSDKQWEQFQTLVQVAHEKLMENGSCELHFLATLAQETGVWKNPVLCTILSQEPLDKDKVNEFLAFEGPILLDMRIKHLIKTNQLSQATALAKLCSDHPEIGIKGSFKQTYLVCLCTSSPNGKLIEEISEVDCKDALEMICNLESEGDEKSALVLCTAFLSRQLQQGDMYCAWELTLFWSKLQQRVEPSIQVYLERCRQLSLLTKTVYHIFFLIKVINSETEGAGLATCIELCVKALRLESTENTEVKISICKTISCLLPDDLEVKRACQLSEFLIEPTVDAYYAVEMLYNQPDQKYDEENLPIPNSLRCELLLVLKTQWPFDPEFWDWKTLKRQCLALMGEEASIVSSIDELNDSEVYEKVVDYQEESKETSMNGLSGGVGANSGLLKDIGDEKQKKREIKQLRERGFISARFRNWQAYMQYCVLCDKEFLGHRIVRHAQKHYKDGIYSCPICAKNFNSKETFVPHVTLHVKQSSKERLAAMKPLRRLGRPPKITTTNENQKTNTVAKQEQRPIKKNSLYSTDFIVFNDNDGSDDENDDKDKSYEPEVIPVQKPVPVNEFNCPVTFCKKGFKYFKNLIAHVKGHKDNEDAKRFLEMQSKKVICQYCRRHFVSVTHLNDHLQMHCGSKPYICIQMKCKAGFNSYAELLTHRKEHQVFRAKCMFPKCGRIFSEAYLLYDHEAQHYNTYTCKFTGCGKVYRSQGELEKHLDDHSTPPEKVLPPEAQLNSSGDSIQPSEVNQNTAENIEKERSMLPSENNIENSLLADRSDAWDKSKAESAVTKQDQISASELRQANGPLSNGLENPATTPLLQSSEVAVSIKVSLNQGIEDNFGKQENSTVEGSGEALVTDLHTPVEDTCNDLCHPGFQER

In [63]:
# ls $botryllus_dir

In [64]:
import screed

for record in screed.open(f"{botryllus_dir}/Bs_proteins.fa"):
    print(record.name)
    print(record.sequence)
    break
bhf_seq = "MVHDTEQLLAQGHHEEETECGKYGKLPEKGSECKKHGIFCRILTALHLKKRRTEHDHQKLLSESQEHLDASTKKTKKKAKKDKRKNKPPKKDSETSKPAQTTISRLPSNRNNNNANSFATTYEKFDNDSLCSVDLIPVDIEFWDMENEPVDQLPHEILESVHMYGDDRFGERLIDRAQNKYAPLDEKQRSESHGAGEYLKHQWKGQGAKKARKRIRTVMKATWQSLQAGARSQTAFLNPQGAVSAALVQNRR"

BHF
MVHDTEQLLAQGHHEEETECGKYGKLPEKGSECKKHGIFCRILTALHLKKRRTEHDHQKLLSESQEHLDASTKKTKKKAKKDKRKNKPPKKDSETSKPAQTTISRLPSNRNNNNANSFATTYEKFDNDSLCSVDLIPVDIEFWDMENEPVDQLPHEILESVHMYGDDRFGERLIDRAQNKYAPLDEKQRSESHGAGEYLKHQWKGQGAKKARKRIRTVMKATWQSLQAGARSQTAFLNPQGAVSAALVQNRR


In [65]:
bhf_hashes["start"] = bhf_hashes.kmer_in_sequence.map(lambda x: bhf_seq.find(x))
bhf_hashes.head(10)

/var/folders/7z/r1593ybs1sj2ks5zzl9vy8840000gn/T/ipykernel_2520/1111197002.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bhf_hashes["start"] = bhf_hashes.kmer_in_sequence.map(lambda x: bhf_seq.find(x))


,kmer_in_sequence,kmer_in_alphabet,hashval,read_name,filename,start
0,VHDTEQLLAQGHHEEETECGKYGK,hppppphhhphpppppppphphhp,1036020595944595459,BHF,Bs_proteins.fa.gz,1
1,TEQLLAQGHHEEETECGKYGKLPE,ppphhhphpppppppphphhphhp,2993707203445337902,BHF,Bs_proteins.fa.gz,4
2,EQLLAQGHHEEETECGKYGKLPEK,pphhhphpppppppphphhphhpp,973051056292048589,BHF,Bs_proteins.fa.gz,5
3,ETECGKYGKLPEKGSECKKHGIFC,pppphphhphhpphpppppphhhp,385192726330471768,BHF,Bs_proteins.fa.gz,16
4,KYGKLPEKGSECKKHGIFCRILTA,phhphhpphpppppphhhpphhph,1194029314525711201,BHF,Bs_proteins.fa.gz,21
5,LPEKGSECKKHGIFCRILTALHLK,hhpphpppppphhhpphhphhphp,3501013581717790569,BHF,Bs_proteins.fa.gz,25
6,EKGSECKKHGIFCRILTALHLKKR,pphpppppphhhpphhphhphppp,723610543814117516,BHF,Bs_proteins.fa.gz,27
7,ECKKHGIFCRILTALHLKKRRTEH,ppppphhhpphhphhphppppppp,2309144907826564485,BHF,Bs_proteins.fa.gz,31
8,CRILTALHLKKRRTEHDHQKLLSE,pphhphhphppppppppppphhpp,1452226584929511062,BHF,Bs_proteins.fa.gz,39
9,LTALHLKKRRTEHDHQKLLSESQE,hphhphppppppppppphhppppp,791636867468234754,BHF,Bs_proteins.fa.gz,42


In [66]:
def get_next_kmer_piece(prev_kmer, next_kmer):
    for i in range(1, len(next_kmer) - 1):
        if prev_kmer.find(next_kmer[:-i]) >= 0:
            return next_kmer[-i:]


def stitch_kmers_together(kmer_hashes, col="kmer_in_sequence"):
    stitched = ""
    for i, row in kmer_hashes.iterrows():
        kmer = row[col]
        if not stitched:
            kmer_piece = kmer
        else:
            kmer_piece = get_next_kmer_piece(stitched, kmer)
        stitched += kmer_piece
    return stitched


folder = "/Users/olgabot/protein-job-search/2024-04-16--botryllus-known-matches"


verbose = False
lines = []

for entry in uniprot_data["results"]:
    accession = entry["primaryAccession"]
    uniprotkbid = entry["uniProtkbId"]
    prefix = f"{accession}__{uniprotkbid}"
    csv = os.path.join(folder, f"{prefix}.csv.gz")
    hashes = read_hash_csv(csv)
    target_matching_hashes = hashes.loc[hashes.hashval.isin(bhf_hashes.hashval)]
    query_hashes_match = bhf_hashes.query("hashval in @target_matching_hashes.hashval")

    if len(target_matching_hashes) == 0:
        continue
    target_hashes_in_query_start = target_matching_hashes.start.min()
    target_hashes_in_query_end = target_matching_hashes.start.max() + ksize

    target_kmer_seq = stitch_kmers_together(target_matching_hashes, "kmer_in_sequence")
    query_kmer_seq = stitch_kmers_together(query_hashes_match, "kmer_in_sequence")
    target_kmer_encoded = stitch_kmers_together(
        target_matching_hashes, "kmer_in_alphabet"
    )
    query_kmer_encoded = stitch_kmers_together(query_hashes_match, "kmer_in_alphabet")

    query_hashes_in_target_start = bhf_seq.find(query_kmer_seq)
    query_hashes_in_target_end = query_hashes_in_target_start + len(query_kmer_seq)
    try:
        assert query_kmer_encoded == target_kmer_encoded
    except AssertionError:
        print(f" query_kmer_encoded: {query_kmer_encoded}")
        print(f"target_kmer_encoded: {target_kmer_encoded}")
        break

    target_seq_match = entry["sequence"]["value"][
        target_hashes_in_query_start:target_hashes_in_query_end
    ]

    # Make sure the sequence in Uniprot matches the sequence from the kmer hashes
    assert target_seq_match == target_kmer_seq

    if verbose:
        print(
            f"\n---\n{prefix} Matches from {target_hashes_in_query_start}-{target_hashes_in_query_end} "
            f"(length: {len(target_seq_match)})"
        )

        print(
            f" target: {target_kmer_seq}  ({target_hashes_in_query_start}-{target_hashes_in_query_end})"
        )
        print(f"encoded: {target_kmer_encoded}")
        print(
            f"  query: {query_kmer_seq} ({query_hashes_in_target_start}-{query_hashes_in_target_end})"
        )

    line = [
        query_hashes_in_target_start,
        query_hashes_in_target_end,
        len(query_kmer_seq),
        query_kmer_seq,
        target_kmer_seq,
        target_kmer_encoded,
        target_hashes_in_query_start,
        target_hashes_in_query_end,
        prefix,
    ]

    n_features = 0
    for feature in get_overlapping_features(
        entry, target_hashes_in_query_start, target_hashes_in_query_end, verbose=verbose
    ):
        if verbose:
            print(feature)
        feature_line = line + [feature["type"], feature["description"]]
        lines.append(feature_line)
        n_features += 1
    if n_features == 0:
        lines.append(line + [None, None])


query_matching_kmers = pd.DataFrame(
    lines,
    columns=[
        "query_start",
        "query_end",
        "length",
        "query_seq",
        "target_seq",
        "target_start",
        "target_end",
        "encoded_seq",
        "name",
        "domain_type",
        "domain_description",
    ],
)
query_matching_kmers = query_matching_kmers.sort_values("query_start")
query_matching_kmers

,query_start,query_end,length,query_seq,target_seq,target_start,target_end,encoded_seq,name,domain_type,domain_description
26,39,66,27,CRILTALHLKKRRTEHDHQKLLSESQE,RRLGRPPKITTTNENQKTNTVAKQEQR,pphhphhphppppppppppphhppppp,601,628,J3KNV1__J3KNV1_HUMAN,Compositional bias,Polar residues
3,39,66,27,CRILTALHLKKRRTEHDHQKLLSESQE,RRLGRPPKITTTNENQKTNTVAKQEQR,pphhphhphppppppppppphhppppp,606,633,O60281__ZN292_HUMAN,Region,Disordered
4,39,66,27,CRILTALHLKKRRTEHDHQKLLSESQE,RRLGRPPKITTTNENQKTNTVAKQEQR,pphhphhphppppppppppphhppppp,606,633,O60281__ZN292_HUMAN,Compositional bias,Polar residues
29,39,66,27,CRILTALHLKKRRTEHDHQKLLSESQE,RRLGRPPKITTTNENQKTNTVAKQEQR,pphhphhphppppppppppphhppppp,606,633,A0A8V8TPI9__A0A8V8TPI9_HUMAN,Region,Disordered
25,39,66,27,CRILTALHLKKRRTEHDHQKLLSESQE,RRLGRPPKITTTNENQKTNTVAKQEQR,pphhphhphppppppppppphhppppp,601,628,J3KNV1__J3KNV1_HUMAN,Region,Disordered
30,39,66,27,CRILTALHLKKRRTEHDHQKLLSESQE,RRLGRPPKITTTNENQKTNTVAKQEQR,pphhphhphppppppppppphhppppp,606,633,A0A8V8TPI9__A0A8V8TPI9_HUMAN,Compositional bias,Polar residues
0,45,73,28,LHLKKRRTEHDHQKLLSESQEHLDASTK,AEGNSKKRKRETEDVPKDKSSTGDGTQN,hphppppppppppphhpppppphphppp,97,125,O43148__MCES_HUMAN,Modified residue,Phosphoserine
1,45,73,28,LHLKKRRTEHDHQKLLSESQEHLDASTK,AEGNSKKRKRETEDVPKDKSSTGDGTQN,hphppppppppppphhpppppphphppp,97,125,O43148__MCES_HUMAN,Mutagenesis,Does not abolish nuclear localization. Abolish...
27,45,73,28,LHLKKRRTEHDHQKLLSESQEHLDASTK,AEGNSKKRKRETEDVPKDKSSTGDGTQN,hphppppppppppphhpppppphphppp,97,125,Q05D80__Q05D80_HUMAN,None,None
28,45,73,28,LHLKKRRTEHDHQKLLSESQEHLDASTK,AEGNSKKRKRETEDVPKDKSSTGDGTQN,hphppppppppppphhpppppphphppp,97,125,Q05DQ2__Q05DQ2_HUMAN,None,None


# ASCII drawing of query protein

In [70]:
feature = query_matching_kmers.iloc[0]
feature

query_start                                    39
query_end                                      66
length                                         27
query_seq             CRILTALHLKKRRTEHDHQKLLSESQE
target_seq            RRLGRPPKITTTNENQKTNTVAKQEQR
target_start          pphhphhphppppppppppphhppppp
target_end                                    601
encoded_seq                                   628
name                         J3KNV1__J3KNV1_HUMAN
domain_type                    Compositional bias
domain_description                 Polar residues
Name: 26, dtype: object

In [71]:
bhf_seq

'MVHDTEQLLAQGHHEEETECGKYGKLPEKGSECKKHGIFCRILTALHLKKRRTEHDHQKLLSESQEHLDASTKKTKKKAKKDKRKNKPPKKDSETSKPAQTTISRLPSNRNNNNANSFATTYEKFDNDSLCSVDLIPVDIEFWDMENEPVDQLPHEILESVHMYGDDRFGERLIDRAQNKYAPLDEKQRSESHGAGEYLKHQWKGQGAKKARKRIRTVMKATWQSLQAGARSQTAFLNPQGAVSAALVQNRR'

In [67]:
# bhf_hashes

In [68]:
matching_hashes.head()

NameError: name 'matching_hashes' is not defined

In [ ]:
stitch_kmers_together(matching_hashes, "kmer_in_sequence")

In [ ]:
stitch_kmers_together(bhf_hashes_match, "kmer_in_alphabet")

In [ ]:
prev_kmer = "LHLKKRRTEHDHQKLLSESQEHLD"
next_kmer = "KKRRTEHDHQKLLSESQEHLDAST"

-1 KKRRTEHDHQKLLSESQEHLDAS
-1 KKRRTEHDHQKLLSESQEHLDA
3 KKRRTEHDHQKLLSESQEHLD
3 KKRRTEHDHQKLLSESQEHL
3 KKRRTEHDHQKLLSESQEH
3 KKRRTEHDHQKLLSESQE
3 KKRRTEHDHQKLLSESQ
3 KKRRTEHDHQKLLSES
3 KKRRTEHDHQKLLSE
3 KKRRTEHDHQKLLS
3 KKRRTEHDHQKLL
3 KKRRTEHDHQKL
3 KKRRTEHDHQK
3 KKRRTEHDHQ
3 KKRRTEHDH
3 KKRRTEHD
3 KKRRTEH
3 KKRRTE
3 KKRRT
3 KKRR
3 KKR
3 KK
